In [1]:
# This command installs all the necessary libraries
!pip install pandas sentence-transformers pillow pinecone tqdm requests scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from PIL import Image
import requests
from io import BytesIO
import pinecone
import numpy as np
from tqdm.auto import tqdm  # tqdm is for the progress bar
import os
from getpass import getpass # To securely ask for your API key
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

In [2]:
# Load the dataset
# !! Update this path to your actual dataset file !!
df = pd.read_csv('intern_data_ikarus.csv')

# Data Cleaning & Preprocessing
# Drop rows where essential information (ID, title, image URL) is missing
df = df.dropna(subset=['uniq_id', 'title', 'images'])

# Fill missing text fields to avoid errors during embedding
df['description'] = df['description'].fillna('No description available')
df['material'] = df['material'].fillna('Unknown')
df['color'] = df['color'].fillna('Unknown')
df['categories'] = df['categories'].fillna('Uncategorized')

print(f"Dataset loaded with {len(df)} products.")
df.head()

Dataset loaded with 312 products.


,title,brand,description,price,categories,images,manufacturer,package_dimensions,country_of_origin,material,color,uniq_id
0,"GOYMFK 1pc Free Standing Shoe Rack, Multi-laye...",GOYMFK,"multiple shoes, coats, hats, and other items E...",$24.99,"['Home & Kitchen', 'Storage & Organization', '...",['https://m.media-amazon.com/images/I/416WaLx1...,GOYMFK,"2.36""D x 7.87""W x 21.6""H",China,Metal,White,02593e81-5c09-5069-8516-b0b29f439ded
1,"subrtex Leather ding Room, Dining Chairs Set o...",subrtex,subrtex Dining chairs Set of 2,NaN,"['Home & Kitchen', 'Furniture', 'Dining Room F...",['https://m.media-amazon.com/images/I/31SejUEW...,Subrtex Houseware INC,"18.5""D x 16""W x 35""H",NaN,Sponge,Black,5938d217-b8c5-5d3e-b1cf-e28e340f292e
2,Plant Repotting Mat MUYETOL Waterproof Transpl...,MUYETOL,No description available,$5.98,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/41RgefVq...,MUYETOL,"26.8""L x 26.8""W",NaN,Polyethylene,Green,b2ede786-3f51-5a45-9a5b-bcf856958cd8
3,"Pickleball Doormat, Welcome Doormat Absorbent ...",VEWETOL,The decorative doormat features a subtle textu...,$13.99,"['Patio, Lawn & Garden', 'Outdoor Décor', 'Doo...",['https://m.media-amazon.com/images/I/61vz1Igl...,Contrence,"24""L x 16""W",NaN,Rubber,A5589,8fd9377b-cfa6-5f10-835c-6b8eca2816b5
4,JOIN IRON Foldable TV Trays for Eating Set of ...,JOIN IRON Store,Set of Four Folding Trays With Matching Storag...,$89.99,"['Home & Kitchen', 'Furniture', 'Game & Recrea...",['https://m.media-amazon.com/images/I/41p4d4VJ...,NaN,"18.9""D x 14.2""W x 26""H",NaN,Iron,Grey Set of 4,bdc9aa30-9439-50dc-8e89-213ea211d66a


In [3]:
# This model is great for general-purpose image and text similarity
model = SentenceTransformer('clip-ViT-B-32')

# This specific CLIP model outputs 512-dimension vectors
embedding_dim = 512

print("CLIP model loaded successfully.")

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


CLIP model loaded successfully.


In [7]:
import json

def get_image_from_url(url):
    """Fetches and opens an image from a URL."""
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status() # Check for bad responses (404, 500, etc.)
        return Image.open(BytesIO(response.content))
    except Exception as e:
        # print(f"Warning: Could not fetch image from {url}. Error: {e}")
        return None

def normalize(v):
    """Normalizes a numpy vector."""
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return v / norm

def parse_image_urls(url_string):
    """
    Parses the 'images' column, which might be a string representation
    of a list, into an actual list of URLs.
    """
    if not isinstance(url_string, str):
        return []

    # Check if it's a stringified list
    if url_string.startswith('[') and url_string.endswith(']'):
        try:
            # Use json.loads for safe parsing of list strings
            url_list = json.loads(url_string.replace("'", "\""))
            return [url.strip() for url in url_list if url]
        except json.JSONDecodeError:
            # Fallback for malformed strings, e.g., '[url1, url2'
            return [url.strip('[]"\' ') for url in url_string.split(',') if url.strip()]

    # Assume it's a single URL
    return [url_string]

print("Starting embedding generation (with multi-image support)...")
vectors_to_upload = []

# Using tqdm for a progress bar
for _, row in tqdm(df.iterrows(), total=df.shape[0]):

    all_embeddings = [] # List to hold all embeddings for this product

    # --- 1. Text Embedding (NLP) ---
    # Create a single descriptive string for the product
    text_desc = (f"Title: {row['title']}. "
                 f"Category: {row['categories']}. "
                 f"Description: {row['description']}. "
                 f"Made of {row['material']} in {row['color']} color.")

    text_emb = model.encode(text_desc)
    all_embeddings.append(normalize(text_emb))

    # --- 2. Image Embeddings (CV) ---
    image_urls = parse_image_urls(row['images'])

    for url in image_urls:
        image = get_image_from_url(url)
        if image:
            try:
                img_emb = model.encode(image)
                all_embeddings.append(normalize(img_emb))
            except Exception as e:
                # print(f"Warning: Could not encode image {url}. Error: {e}")
                pass # Skip this one image

    # --- 3. Combine All Embeddings ---
    if len(all_embeddings) > 0:
        # Average all normalized vectors (text + all images)
        final_vector = np.mean(all_embeddings, axis=0)
    else:
        # Should not happen if text embedding worked, but as a fallback
        continue # Skip this product if no embeddings could be generated

    # --- 4. Store for Upload ---
    # Get the *first* image URL for the metadata preview
    first_image_url = image_urls[0] if image_urls else "No image"

    metadata = {
        'title': str(row['title'])[:500],
        'image_url': first_image_url, # Just for display
        'price': str(row['price']),
        'categories': str(row['categories'])
    }

    vectors_to_upload.append({
        "id": str(row['uniq_id']),
        "values": final_vector.tolist(), # Convert numpy array to list
        "metadata": metadata
    })

print(f"\nSuccessfully generated {len(vectors_to_upload)} embeddings.")

Starting embedding generation (with multi-image support)...


  0%|          | 0/312 [00:00<?, ?it/s]


Successfully generated 312 embeddings.


In [8]:
import os
from pinecone import Pinecone, ServerlessSpec
from getpass import getpass

# 1. Get API key securely and set it as an environment variable
PINECONE_API_KEY = getpass("Enter your Pinecone API Key: ")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

# 2. Initialize Pinecone
# Pinecone() will automatically read the API key from the environment variable
pc = Pinecone()

index_name = "product-recommender-clip"
# We know this from Step 4, our CLIP model outputs 512 dimensions
embedding_dim = 512

# 3. Check if index exists, and if not, create it
if index_name not in pc.list_indexes().names():
    print(f"Creating new serverless index: {index_name}")
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine", # Cosine is great for normalized CLIP vectors
        spec=ServerlessSpec(
            cloud="aws",       # Or "gcp", "azure"
            region="us-east-1" # Choose a region
        )
    )
    print(f"Created new Pinecone index: {index_name}")
else:
    print(f"Using existing Pinecone index: {index_name}")

# 4. Get the index object
index = pc.Index(index_name)

# 5. Upload vectors in batches (This part is the same as before)
# We use index.upsert() because we have already pre-computed our vectors
batch_size = 100
print(f"Uploading vectors in batches of {batch_size}...")

# 'vectors_to_upload' is the list we generated in Step 5
for i in tqdm(range(0, len(vectors_to_upload), batch_size)):
    batch = vectors_to_upload[i:i + batch_size]
    try:
        index.upsert(vectors=batch)
    except Exception as e:
        print(f"Error uploading batch {i}-{i+batch_size}: {e}")

print("\nUpload complete.")
print("Index stats:")
# Print the new stats object
print(index.describe_index_stats())

Enter your Pinecone API Key: ··········
Creating new serverless index: product-recommender-clip
Created new Pinecone index: product-recommender-clip
Uploading vectors in batches of 100...


  0%|          | 0/4 [00:00<?, ?it/s]


Upload complete.
Index stats:
{'dimension': 512,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}
